<a href="https://colab.research.google.com/github/RUMONMD89/Computing-science-with-Fortran/blob/main/ACS15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/FORTRAN

Mounted at /content/drive
/content/drive/MyDrive/FORTRAN


#MPI

In [2]:
%%writefile MPI.f03
! Sample program of Messge Passing Interface (MPI)
! For simplicity, data is not split (only calculation is split)
program main
  use mpi
  implicit none
  integer, parameter :: IDIM=100
  double precision   :: a(IDIM), x(IDIM), b, c, aa(IDIM)
  integer            :: ista, iend, i, ndim
  integer            :: ierr, nprocs, myrank
!
  a(:)  = 0.0d0
  aa(:) = 0.0d0
  x(:)  = 1.0d0
  b     = 2.0d0
  c     = 3.0d0
!
  call MPI_Init(ierr)
  call MPI_Comm_Size(MPI_COMM_WORLD, nprocs, ierr)
  call MPI_Comm_Rank(MPI_COMM_WORLD, myrank, ierr)
!
! Set the range of index
  ndim = IDIM/nprocs
  ista = 1 + myrank*ndim
  iend = ista + ndim - 1
!
  write(*,*) "myrank=",myrank,"ista=",ista," iend=",iend
!
  do i = ista, iend
    a(i) = b*x(i) + c
  enddo
!
  call MPI_Allreduce(a, aa, IDIM, MPI_DOUBLE_PRECISION, &
 &   MPI_MAX, MPI_COMM_WORLD, ierr)
!
  if(myrank == 0) then
    write(*,*) "aa(:)=",aa(:)
  endif
!
  call MPI_Barrier(MPI_COMM_WORLD, ierr)
  call MPI_Finalize(ierr)
end program main

Writing MPI.f03


In [3]:
EXE="MPI"
OMPI_ALLOW_RUN_AS_ROOT=1
OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1

!mpif90 {EXE}.f03 -o ./{EXE}.out
!mpirun --allow-run-as-root --oversubscribe -np 2 ./{EXE}.out

 myrank=           1 ista=          51  iend=         100
 myrank=           0 ista=           1  iend=          50
 aa(:)=   5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.00000000000000

#OpenMP

In [4]:
%%writefile OpenMP.f03
! Sample program of OpenMP
program main
  use omp_lib
  implicit none
  integer, parameter :: IDIM=100
  double precision   :: a(IDIM), x(IDIM), b, c
  integer            :: i
  integer            :: numthread, ithread
!
  a(:)  = 0.0d0
  x(:)  = 1.0d0
  b     = 2.0d0
  c     = 3.0d0
!
! Get number of threads
  numthread = omp_get_max_threads()
  ithread   = omp_get_thread_num()
  write(*,*) "ithread=",ithread,"numthread=",numthread
!
!$OMP Parallel Default(shared)
!$OMP Do
  do i = 1, IDIM
    a(i) = b*x(i) + c
  enddo
!$OMP End Parallel
!
  write(*,*) "a(:)=",a(:)
!
end program main

Writing OpenMP.f03


In [5]:
EXE="OpenMP"
OMP_NUM_THREADS=2

!gfortran {EXE}.f03 -fopenmp -o ./{EXE}.out
!./{EXE}.out

 ithread=           0 numthread=           2
 a(:)=   5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.00000000

#OpenACC

In [6]:
%%writefile OpenACC.f03
! Sample program of OpenMP
program main
  use omp_lib
  implicit none
  integer, parameter :: IDIM=100
  double precision   :: a(IDIM), x(IDIM), b, c
  integer            :: i
  integer            :: numthread, ithread
!
  a(:)  = 0.0d0
  x(:)  = 1.0d0
  b     = 2.0d0
  c     = 3.0d0
!
! Get number of threads
  numthread = omp_get_max_threads()
  ithread   = omp_get_thread_num()
  write(*,*) "ithread=",ithread,"numthread=",numthread
!
!$acc kernels copy(a) copyin(b,x,c)
!$acc loop private(i)
  do i = 1, IDIM
    a(i) = b*x(i) + c
  enddo
!$acc end kernels
!
  write(*,*) "a(:)=",a(:)
!
end program main

Writing OpenACC.f03


In [7]:
EXE="OpenACC"

!gfortran {EXE}.f03 -fopenacc -o ./{EXE}.out
!./{EXE}.out

 ithread=           0 numthread=           2
 a(:)=   5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.0000000000000000        5.00000000